In [1]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os


# Using energy Source Datasets

# Source of Energy Petroleum

In [2]:
# Read the Excel file
# Data_Status: Indicates the status of the data. The value "2020F" suggests that it is a forecast for the year 2020.
# State: Represents the state for which the data is recorded (in this case, "CA" for California).
# MSN: Stands for "Monthly State Names" and refers to the specific energy metric or variable being measured. Examples include ARICD, ARICV, ARTCD, ARTCV, ARTXD, WWTXV, WXICD, WXICV, ZWCDP, ZWHDP.
df = pd.read_excel('Datasets/use_energy_source.xlsx' , sheet_name='Petroleum')


In [3]:
df.head()

,State,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,45410,51008,53752,55429,58130,61705,65655,76363,80412,...,269021,250727,231561,226262,236966,219904,216757,216054,221066,222491
1,AL,236010,232362,239387,251791,256646,261907,272450,277348,306563,...,521527,513751,492104,491040,512064,538768,530813,518058,529090,525533
2,AR,148055,159835,166202,172970,184126,178998,194561,200031,219831,...,337975,320965,320684,325487,314418,322423,321405,325863,330176,315085
3,AZ,121344,126775,135742,144241,147476,145724,159220,167963,188622,...,553940,540510,546616,545383,561231,579022,586125,592832,609512,547094
4,CA,1836912,1892977,1879327,1915789,2027468,2089816,2219598,2251677,2403681,...,3180967,3085251,3104484,3090739,3201730,3316932,3392631,3406056,3385576,2672900


# Data statistics

In [4]:
df.head()

,State,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,45410,51008,53752,55429,58130,61705,65655,76363,80412,...,269021,250727,231561,226262,236966,219904,216757,216054,221066,222491
1,AL,236010,232362,239387,251791,256646,261907,272450,277348,306563,...,521527,513751,492104,491040,512064,538768,530813,518058,529090,525533
2,AR,148055,159835,166202,172970,184126,178998,194561,200031,219831,...,337975,320965,320684,325487,314418,322423,321405,325863,330176,315085
3,AZ,121344,126775,135742,144241,147476,145724,159220,167963,188622,...,553940,540510,546616,545383,561231,579022,586125,592832,609512,547094
4,CA,1836912,1892977,1879327,1915789,2027468,2089816,2219598,2251677,2403681,...,3180967,3085251,3104484,3090739,3201730,3316932,3392631,3406056,3385576,2672900


In [5]:
df.describe()

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
count,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,...,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01
mean,7.643712e+05,7.756938e+05,8.075954e+05,8.324804e+05,8.553942e+05,8.916909e+05,9.359399e+05,9.720199e+05,1.037110e+06,1.089293e+06,...,1.332249e+06,1.301278e+06,1.324188e+06,1.334112e+06,1.361627e+06,1.374816e+06,1.387408e+06,1.419675e+06,1.418656e+06,1.244179e+06
std,2.734543e+06,2.775780e+06,2.888954e+06,2.978265e+06,3.060821e+06,3.191692e+06,3.350099e+06,3.479418e+06,3.711415e+06,3.898068e+06,...,4.787576e+06,4.679947e+06,4.766435e+06,4.800208e+06,4.902050e+06,4.952331e+06,4.999775e+06,5.121277e+06,5.120199e+06,4.491973e+06
min,4.541000e+04,5.046300e+04,5.247300e+04,5.251900e+04,4.957500e+04,5.890900e+04,6.428400e+04,6.373100e+04,6.764500e+04,6.975400e+04,...,2.225200e+04,1.938500e+04,1.881500e+04,2.015200e+04,2.042600e+04,1.950800e+04,1.692700e+04,1.902500e+04,1.861100e+04,1.553500e+04
25%,1.232910e+05,1.243798e+05,1.325378e+05,1.294985e+05,1.293358e+05,1.326062e+05,1.394505e+05,1.458230e+05,1.491288e+05,1.585305e+05,...,2.324055e+05,2.332345e+05,2.315315e+05,2.347552e+05,2.364198e+05,2.297702e+05,2.296232e+05,2.344920e+05,2.381218e+05,2.137560e+05
50%,2.321245e+05,2.344435e+05,2.474960e+05,2.575610e+05,2.596640e+05,2.693745e+05,2.805140e+05,2.874115e+05,3.129575e+05,3.392570e+05,...,4.843030e+05,4.705785e+05,4.811010e+05,4.845185e+05,5.061950e+05,5.032100e+05,5.083935e+05,5.165605e+05,5.228825e+05,4.448935e+05
75%,4.484248e+05,4.482260e+05,4.693982e+05,4.962358e+05,5.144208e+05,5.444295e+05,5.665190e+05,5.813122e+05,6.142072e+05,6.731878e+05,...,7.964492e+05,7.835125e+05,8.015285e+05,8.340880e+05,8.431272e+05,8.560880e+05,8.524545e+05,8.780195e+05,8.534085e+05,7.031342e+05
max,1.987365e+07,2.016804e+07,2.099748e+07,2.164449e+07,2.224025e+07,2.318396e+07,2.433444e+07,2.527252e+07,2.696486e+07,2.832163e+07,...,3.463846e+07,3.383323e+07,3.442890e+07,3.468691e+07,3.540229e+07,3.574521e+07,3.607261e+07,3.691154e+07,3.688506e+07,3.234865e+07


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 62 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   State   52 non-null     object
 1   1960    52 non-null     int64 
 2   1961    52 non-null     int64 
 3   1962    52 non-null     int64 
 4   1963    52 non-null     int64 
 5   1964    52 non-null     int64 
 6   1965    52 non-null     int64 
 7   1966    52 non-null     int64 
 8   1967    52 non-null     int64 
 9   1968    52 non-null     int64 
 10  1969    52 non-null     int64 
 11  1970    52 non-null     int64 
 12  1971    52 non-null     int64 
 13  1972    52 non-null     int64 
 14  1973    52 non-null     int64 
 15  1974    52 non-null     int64 
 16  1975    52 non-null     int64 
 17  1976    52 non-null     int64 
 18  1977    52 non-null     int64 
 19  1978    52 non-null     int64 
 20  1979    52 non-null     int64 
 21  1980    52 non-null     int64 
 22  1981    52 non-null     int6

In [7]:
df.head()

,State,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,45410,51008,53752,55429,58130,61705,65655,76363,80412,...,269021,250727,231561,226262,236966,219904,216757,216054,221066,222491
1,AL,236010,232362,239387,251791,256646,261907,272450,277348,306563,...,521527,513751,492104,491040,512064,538768,530813,518058,529090,525533
2,AR,148055,159835,166202,172970,184126,178998,194561,200031,219831,...,337975,320965,320684,325487,314418,322423,321405,325863,330176,315085
3,AZ,121344,126775,135742,144241,147476,145724,159220,167963,188622,...,553940,540510,546616,545383,561231,579022,586125,592832,609512,547094
4,CA,1836912,1892977,1879327,1915789,2027468,2089816,2219598,2251677,2403681,...,3180967,3085251,3104484,3090739,3201730,3316932,3392631,3406056,3385576,2672900


# Transformation of Data

In [8]:
df_trans = df.melt(id_vars=['State'] , var_name='Year', value_name='Yearly Data')
df_trans['Year'] = pd.to_datetime(df_trans['Year'], format='%Y')

df_trans


,State,Year,Yearly Data
0,AK,1960-01-01,45410
1,AL,1960-01-01,236010
2,AR,1960-01-01,148055
3,AZ,1960-01-01,121344
4,CA,1960-01-01,1836912
...,...,...,...
3167,WA,2020-01-01,623049
3168,WI,2020-01-01,494034
3169,WV,2020-01-01,180570
3170,WY,2020-01-01,139779


# Modeling the Data
## Using ARIMA Model

In [16]:
os.makedirs('Plots/use_energy_source_Petroleum/Arima_results_plots',exist_ok=True)

for State in df_trans['State'].unique():
        try:
            
            fig = go.Figure()

            # Get the energy consumption data for the current country and sector
            df_filter = df_trans[df_trans['State'] == State ][['Year', 'Yearly Data']]
            df_filter_index = df_filter.set_index('Year')

            train_data = df_filter[:-5]
            test_data = df_filter[-5:]
            
            # Prepare the data for modeling
            years = df_filter_index.index
            energy_consumption = df_filter_index.values.flatten()

                    # Split the data into training and testing
            # Use all data except the last 5 years for training
            Horizan = -5
            train_data = energy_consumption[:Horizan]
            test_data = energy_consumption[Horizan:]  # Use the last 5 years for testing

            # Fit the auto ARIMA model
            model = auto_arima(train_data, seasonal=False)
            model.fit(train_data)

            # Generate predictions
            predictions = model.predict(n_periods=len(test_data))
            predictions_ahead_in_future = model.predict(n_periods=len(test_data)+15)

            # Calculate evaluation metrics
            mae = mean_absolute_error(test_data, predictions)
            mse = mean_squared_error(test_data, predictions)
            mape = np.mean(np.abs((test_data - predictions) / test_data)) * 100

            print('Mean Absolute Error (MAE):', np.round(mae,2))
            print('Mean Squared Error (MSE):', np.round(mse,2))
            print('Mean Absolute Percentage Error (MAPE):', np.round(mape,2))
            
            # Plot the training data
            fig.add_trace(go.Scatter(
                x=years[:Horizan], y=train_data, mode='lines+markers', name='Training Data'))

            # Plot the predictions
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=test_data, mode='lines+markers', name='Actual'))
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=predictions, mode='lines+markers', name='Predicted'))

            fig.add_trace(go.Scatter(
                x=pd.date_range(start = years[Horizan],periods=15,freq='Y'), y=predictions_ahead_in_future, mode='lines+markers', name='Prediction till 2030'))

            # Update the layout
            fig.update_layout(title=f'Coal : Energy Consumption Forecast State : {State} using ARIMA Model',
                            xaxis_title='Year', yaxis_title='Energy Consumption')

            # Show the plot
            # fig.show()
            # print(State,msn)
            fig.write_image(f'Plots/use_energy_source_Petroleum/Arima_results_plots/{State}.png')
            # break
        except:
            print('Error occoured in Combination State : {} and MSN : {} Due NaN Value'.format(State,mse))
        

Mean Absolute Error (MAE): 21269.49
Mean Squared Error (MSE): 458522966.32
Mean Absolute Percentage Error (MAPE): 9.71
Mean Absolute Error (MAE): 13577.71
Mean Squared Error (MSE): 230145688.21
Mean Absolute Percentage Error (MAPE): 2.55
Mean Absolute Error (MAE): 12594.5
Mean Squared Error (MSE): 183664434.9
Mean Absolute Percentage Error (MAPE): 3.88
Mean Absolute Error (MAE): 19869.33
Mean Squared Error (MSE): 601107161.07
Mean Absolute Percentage Error (MAPE): 3.48
Mean Absolute Error (MAE): 244621.0
Mean Squared Error (MSE): 80984865499.4
Mean Absolute Percentage Error (MAPE): 8.06
Mean Absolute Error (MAE): 22743.93
Mean Squared Error (MSE): 1175489437.94
Mean Absolute Percentage Error (MAPE): 4.88
Mean Absolute Error (MAE): 19195.2
Mean Squared Error (MSE): 648230733.2
Mean Absolute Percentage Error (MAPE): 6.88
Mean Absolute Error (MAE): 2504.8
Mean Squared Error (MSE): 8452926.4
Mean Absolute Percentage Error (MAPE): 14.8
Mean Absolute Error (MAE): 6225.0
Mean Squared Error (M

## Using SARIMA Model

In [9]:
os.makedirs('Plots/use_energy_source_Petroleum/Sarima_results_plots',exist_ok=True)

for State in df_trans['State'].unique():
        try:
            
            fig = go.Figure()

            # Get the energy consumption data for the current country and sector
            df_filter = df_trans[df_trans['State'] == State ][['Year', 'Yearly Data']]
            df_filter_index = df_filter.set_index('Year')

            train_data = df_filter[:-5]
            test_data = df_filter[-5:]
            
            # Prepare the data for modeling
            years = df_filter_index.index
            energy_consumption = df_filter_index.values.flatten()

                    # Split the data into training and testing
            # Use all data except the last 5 years for training
            Horizan = -5
            train_data = energy_consumption[:Horizan]
            test_data = energy_consumption[Horizan:]  # Use the last 5 years for testing

            # Fit the auto ARIMA model
            model = auto_arima(train_data, seasonal=False)
            model.fit(train_data)

            # Generate predictions
            predictions = model.predict(n_periods=len(test_data))
            predictions_ahead_in_future = model.predict(n_periods=len(test_data)+15)

            # Calculate evaluation metrics
            mae = mean_absolute_error(test_data, predictions)
            mse = mean_squared_error(test_data, predictions)
            mape = np.mean(np.abs((test_data - predictions) / test_data)) * 100

            print('Mean Absolute Error (MAE):', np.round(mae,2))
            print('Mean Squared Error (MSE):', np.round(mse,2))
            print('Mean Absolute Percentage Error (MAPE):', np.round(mape,2))
            
            # Plot the training data
            fig.add_trace(go.Scatter(
                x=years[:Horizan], y=train_data, mode='lines+markers', name='Training Data'))

            # Plot the predictions
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=test_data, mode='lines+markers', name='Actual'))
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=predictions, mode='lines+markers', name='Predicted'))

            fig.add_trace(go.Scatter(
                x=pd.date_range(start = years[Horizan],periods=15,freq='Y'), y=predictions_ahead_in_future, mode='lines+markers', name='Prediction till 2030'))

            # Update the layout
            fig.update_layout(title=f'Coal : Energy Consumption Forecast State : {State} using SARIMA Model',
                            xaxis_title='Year', yaxis_title='Energy Consumption')

            # Show the plot
            # fig.show()
            # print(State,msn)
            fig.write_image(f'Plots/use_energy_source_Petroleum/Sarima_results_plots/{State}.png')
            # break
        except:
            print('Error occoured in Combination State : {} and MSN : {} Due NaN Value'.format(State,mse))
        # break

Mean Absolute Error (MAE): 21269.49
Mean Squared Error (MSE): 458522966.32
Mean Absolute Percentage Error (MAPE): 9.71
Mean Absolute Error (MAE): 13577.71
Mean Squared Error (MSE): 230145688.21
Mean Absolute Percentage Error (MAPE): 2.55
Mean Absolute Error (MAE): 12594.5
Mean Squared Error (MSE): 183664434.9
Mean Absolute Percentage Error (MAPE): 3.88
Mean Absolute Error (MAE): 19869.33
Mean Squared Error (MSE): 601107161.07
Mean Absolute Percentage Error (MAPE): 3.48
Mean Absolute Error (MAE): 244621.0
Mean Squared Error (MSE): 80984865499.4
Mean Absolute Percentage Error (MAPE): 8.06
Mean Absolute Error (MAE): 22743.93
Mean Squared Error (MSE): 1175489437.94
Mean Absolute Percentage Error (MAPE): 4.88
Mean Absolute Error (MAE): 19195.2
Mean Squared Error (MSE): 648230733.2
Mean Absolute Percentage Error (MAPE): 6.88
Mean Absolute Error (MAE): 2504.8
Mean Squared Error (MSE): 8452926.4
Mean Absolute Percentage Error (MAPE): 14.8
Mean Absolute Error (MAE): 6225.0
Mean Squared Error (M